In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import seaborn as sn
import matplotlib.pyplot as plt

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from gensim.models import doc2vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import spacy

In [ ]:
from tqdm import tqdm
import random

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install contextualSpellCheck

In [ ]:
import contextualSpellCheck

In [ ]:
!pip install autocorrect

In [ ]:
from autocorrect import Speller

In [ ]:
from spacy.matcher import Matcher

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Fake reviews detection/chi_hotel_dataset.csv')

In [ ]:
#df = pd.read_csv('restaurant_reviews_anonymized.csv', encoding='ISO-8859-1')

In [ ]:
df.head()

,Real,Review
0,1,Let me begin by saying that there are two kind...
1,1,The only place inside the Loop that you can st...
2,1,I have walked by the Tokyo Hotel countless tim...
3,1,"If you are considering staying here, watch thi..."
4,1,"This place is disgusting, absolutely horrible,..."


In [ ]:
#df.rename(columns = {'Reviews':'Review'}, inplace = True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5854 entries, 0 to 5853
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Real    5854 non-null   int64 
 1   Review  5854 non-null   object
dtypes: int64(1), object(1)
memory usage: 91.6+ KB


In [ ]:
#df = df[~df['Review'].isnull()]

In [ ]:
#df.reset_index(inplace=True)

## Feature extraction

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
spell = Speller(lang='en')

In [ ]:
avg_word_length = []
pausality = []
avg_np = []
avg_sent_length = []
num_clauses = []
num_words = []
num_verbs = []
num_adj = []
num_pv = []
emotiveness = []
content_div = []
redundancy = []
lexical_div = []
num_mv = []
num_typos = []

In [ ]:
for i in tqdm(range(len(df))):
    doc = nlp(df['Review'][i].replace('\r', ''))
    n_adj = 0
    n_verbs = 0
    n_nouns = 0
    n_adv = 0
    n_passive = 0
    n_mv = 0
    n_cl = 0
    n_punc = 0
    redun = 0
    n_typos = 0

    for token in doc:
        if token.pos_ == 'ADJ':
            n_adj += 1
        elif token.pos_ == 'VERB':
            n_verbs += 1
        elif token.pos_ == 'NOUN':
            n_nouns += 1
        elif token.pos_ == 'ADV':
            n_adv += 1
        if token.dep_ == 'aux' and token.tag_ == 'MD':
            #print(token)
            n_mv += 1
        if token.dep_ == 'nsubjpass':
            #print(token)
            n_passive += 1
        elif token.dep_ == 'ROOT':
            #print(token)
            n_cl += 1
        if token.is_punct == True:
            n_punc += 1

    words = [token.text.lower() for token in doc if token.is_punct == False]
    len_words = 0

    filtered = []

    for word in words:
        len_words += len(word)
        if spell(word) != word:
            n_typos += 1
        if not word in stop_words:
            filtered.append(word)

    if len(words) != 0:
        avg_wl = len_words/len(words)
    else:
        avg_wl = 0

    if len(list(doc.sents)) != 0:
        avg_sl = len(doc)/len(list(doc.sents)) 
    else:
        avg_sl = 0

    if n_nouns + n_verbs != 0:
        emo = (n_adj + n_adv)/(n_nouns + n_verbs)
    else:
        emo = 0  

    if len(words) != 0:
        lxd = len(set(words))/len(words)
    else:
        lxd = 0

    if len(words) != 0:
        cdv = len(set(filtered))/len(words)
    else:
        cdv = 0   

    len_chunk = 0
    for chunk in doc.noun_chunks:
        len_chunk += len(chunk)

    ll = len([chunk.text for chunk in doc.noun_chunks])    
    if ll != 0:
        anp = len_chunk/ll
    else:
        anp = 0

    if len(list(doc.sents)) != 0:
        redun = (len(words) - len(filtered))/len(list(doc.sents))
    else:
        redun = 0


    avg_word_length.append(avg_wl)
    pausality.append(n_punc/len(list(doc.sents)))
    avg_np.append(anp)
    avg_sent_length.append(avg_sl)
    num_clauses.append(n_cl)
    num_words.append(len(words))
    num_verbs.append(n_verbs)
    num_adj.append(n_adj)
    num_pv.append(n_passive)
    emotiveness.append(emo)
    content_div.append(cdv)
    redundancy.append(redun)
    lexical_div.append(lxd)
    num_mv.append(n_mv)
    num_typos.append(n_typos)


100%|██████████| 5854/5854 [20:04<00:00,  4.86it/s]


In [ ]:
# print('Av WL:', avg_wl,'\nPausality:', pausality, '\naverage NP:', anp,'\nSL:', avg_sl,'\nCl:', n_cl,'\nWords:', len(words), '\nVerbs:', n_verbs,
#       '\nAdj:', n_adj,'\nPassive:', n_passive, '\nEmo:', emo,'\nCDV:', cdv,'\nRedun:', redun, '\nLXD:', lxd, '\nMV:',n_mv, '\nTypos:', n_typos)

In [ ]:
columns = ['Average word length', 'Pausality', 'Average NP length', 'Average sentence length', 'Number of clauses', 'Number of words', 'Number of verbs', 
           'Number of adjectives', 'Number of passive voice', 'Emotiveness', 'Content diversity', 'Redundancy', 'Lexical diversity', 'Number of modal verbs', 'Number of typos']

In [ ]:
df_features = pd.DataFrame(list(zip(avg_word_length, pausality, avg_np, avg_sent_length, num_clauses, num_words, num_verbs, 
                                   num_adj, num_pv, emotiveness, content_div, redundancy, lexical_div, num_mv, num_typos)), columns = columns)

In [ ]:
df_features.to_csv('chi_hotel_features.csv', index=False)